<a href="https://www.kaggle.com/code/karelbecerra/inference-enhanced-hand-data?scriptVersionId=212917497" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

# Notebook environment setup
> This notebook is for the intention of having an easy start process. You can find more '**production ready**' code for **training** and **inference** into **/classification** repository folder: [GitHub Repository](https://github.com/karelbecerra/sam-x-ray-medical-images-hand-sex-classification)


**Notebook requeriments**:
* Install public packages
* Import public packages

**Model**:
* Re use Training notebook code
* Setup working folders

**Inference**:
* Load parameters
* Load pre-trained model
* Load dataset
* Run inference
* Plot confusion matrix

***

### Notebook requeriments

In [ ]:
# Import public packages
import os
import sys
import torch
import numpy as np

In [ ]:
# Install public packages
!pip install torchsummary
!pip install munch

***

### Model & Code
There are 2 options:
* Clone github code [repository](https://github.com/karelbecerra/sam-x-ray-medical-images-hand-sex-classification)
* Inherit code from [Training](https://www.kaggle.com/code/karelbecerra/training-enhanced-hand-data) notebook.

In this notebook we will re use **Training** notebook. Therefore '**pre-trained model**' will be in notebook **output** folder as **enhanced_hand_data_run1.pth**

In [ ]:
# Setup working folders to Training notebook output
troncho_base = '/kaggle/input/training-enhanced-hand-data/classification'
os.chdir(troncho_base)
sys.path.append(troncho_base)

# List code folders and file structure
os.listdir('./')

In [ ]:
# List all pre-trained models
os.listdir('./output/models')

***
## Inference

In [ ]:
from src.utils import config
from src.loaddataset import load_dataset_for_inference
from src.buildmodel import build_model
from src.utils.modelfile import load_model

In [ ]:
# Load parameters
parameters = config.load_parameters('enhanced_hand_data')

# Build model (will load model architecture and default EfficientNet weights)
model, device = build_model(parameters)

# Load trained weights
load_model(parameters=parameters, model=model, model_file='enhanced_hand_data_run1.pth')

In [ ]:
# Load dataset
test_dataset = '/kaggle/input/sam-x-ray-medical-images-hand-sex-classification/enhanced_hand_data/tst_masked_he_400x300.npz'
test_loader = load_dataset_for_inference(data_file=test_dataset,  batch_size=600)

In [ ]:
# Turn model to evaluation state
model.eval()

with torch.no_grad():
    for i, data in enumerate(test_loader):
        x, target = data
        x = x.to(device)
        out = model(x)
        x = x.cpu().numpy()
        target = target.cpu().numpy()
        out = out.cpu().numpy()
        y_data = np.concatenate((y_data, target), axis=0) if i != 0 else target
        out_data = np.concatenate((out_data, out), axis=0) if i != 0 else out


In [ ]:
from stats.confusion_matrix import get_confusion_matrix, plot_confusion_matrix

# Create confusion matrix from ground thruth (y_data) vs prediction (out_data)
confusion_matrix, _, _ = get_confusion_matrix(y_data, out_data)

# Plot
plot_confusion_matrix( confusion_matrix )